In [3]:
import numpy as np
import pandas as pd
import csv
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import os
import re
import pandas as pd
import numpy as np
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import nltk
from nltk.stem import WordNetLemmatizer 
# !pip3 install pyspellchecker
from spellchecker import SpellChecker
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


Defaulting to user installation because normal site-packages is not writeable


# Data preparation

In [4]:
class Preprocessing:
    def __init__(self, path_to_data):
        files = []
        tweets = pd.DataFrame()
        for filenames in os.listdir(path_to_data):
            if re.findall('.+\.csv', filenames):
                files.append(filenames)
        for file in files:
            with open(path_to_data + '/' + file, 'r') as f:
                data = f.read()
            data = re.sub(r',(?=[^ ])', '--', data)
            df = pd.DataFrame(data.split('--'), columns=['tweets'])
            df['category'] = file
            df.drop_duplicates(inplace=True)
            tweets = tweets.append(df)
        tweets['category'].replace({
            'processedNegative.csv' : -1, 
            'processedNeutral.csv': 0, 
            'processedPositive.csv' : 1}, inplace=True)
        tweets.index = np.arange(len(tweets))
        self.tweets = tweets
        self.spell = SpellChecker()


    def tokenization(self):
        array = []
        for sent in self.tweets.tweets:
            array.append(word_tokenize(sent))
        self.tweets["just_tokenization"] = array

    def PorterStemmer(self, misspellings=False):
        array = []
        porter_stemmer = PorterStemmer()
        for sent in self.tweets.tweets:
            array.append(word_tokenize(sent))
            for i, w in enumerate(array[-1]):
                if (misspellings == True):
                    array[-1][i] = self.spell.correction(w)
                array[-1][i] = porter_stemmer.stem(w)
    
        if (misspellings):
            self.tweets["PorterStemmer+misspellings"] = array
        else:
            self.tweets["PorterStemmer"] = array

    def WordNetLemmatizer(self, misspellings=False):
        nltk.download('wordnet')
        array = []
        lemmatizer = WordNetLemmatizer()
        for sent in self.tweets.tweets:
            array.append(word_tokenize(sent))
            for i, w in enumerate(array[-1]):
                if (misspellings == True):
                    array[-1][i] = self.spell.correction(w)
                array[-1][i] = lemmatizer.lemmatize(w)
                    
        if (misspellings):
            self.tweets["WordNetLemmatizer+misspellings"] = array
        else:
            self.tweets["WordNetLemmatizer"] = array

    # def our preprocessing:

        #     def remove_stopwords(text):
        # 	tokens = word_tokenize(text.lower())
        # 	english_stopwords = stopwords.words('english')
        # 	text = [w for w in tokens if w not in english_stopwords]
        # 	return " ".join(text)

        # def clear_df(text)->str:
        # 	# text = text.lower()
        # 	text = contractions.fix(text)
        # 	text = re.sub(r'http\S+', '', text)
        # 	text = re.sub(r'[^\w\s]', '', text)
        # 	text = remove_stopwords(text)

        # 	return text

        # def snowball_stemmer(text):
        # 	stemmer = SnowballStemmer(language='english')
        # 	tokens = word_tokenize(text.lower())
        # 	stem_words = []
        # 	for w in tokens:
        # 		x = stemmer.stem(w)
        # 		stem_words.append(x)
        # 	return " ".join(stem_words)


In [ ]:
#минут 6 учицца - норма
preproc = Preprocessing("./data")
preproc.tokenization()
preproc.PorterStemmer(False)
preproc.WordNetLemmatizer(False)
preproc.WordNetLemmatizer(True)
preproc.PorterStemmer(True)
preproc_data = preproc.tweets
preproc_data

In [17]:
preproc_data

,tweets,category,just_tokenization,PorterStemmer+misspellings,WordNetLemmatizer+misspellings,PorterStemmer,WordNetLemmatizer
0,How unhappy some dogs like it though,-1,"[How, unhappy, some, dogs, like, it, though]","[how, unhappi, some, dog, like, it, though]","[How, unhappy, some, dog, like, it, though]","[how, unhappi, some, dog, like, it, though]","[How, unhappy, some, dog, like, it, though]"
1,talking to my over driver about where I'm goin...,-1,"[talking, to, my, over, driver, about, where, ...","[talk, to, my, over, driver, about, where, i, ...","[talking, to, my, over, driver, about, where, ...","[talk, to, my, over, driver, about, where, i, ...","[talking, to, my, over, driver, about, where, ..."
2,Does anybody know if the Rand's likely to fall...,-1,"[Does, anybody, know, if, the, Rand, 's, likel...","[doe, anybodi, know, if, the, rand, 's, like, ...","[Does, anybody, know, if, the, Rand, 's, likel...","[doe, anybodi, know, if, the, rand, 's, like, ...","[Does, anybody, know, if, the, Rand, 's, likel..."
3,I miss going to gigs in Liverpool unhappy,-1,"[I, miss, going, to, gigs, in, Liverpool, unha...","[i, miss, go, to, gig, in, liverpool, unhappi]","[I, miss, going, to, gig, in, Liverpool, unhappy]","[i, miss, go, to, gig, in, liverpool, unhappi]","[I, miss, going, to, gig, in, Liverpool, unhappy]"
4,There isnt a new Riverdale tonight ? unhappy,-1,"[There, isnt, a, new, Riverdale, tonight, ?, u...","[there, isnt, a, new, riverdal, tonight, ?, un...","[There, isnt, a, new, Riverdale, tonight, ?, u...","[there, isnt, a, new, riverdal, tonight, ?, un...","[There, isnt, a, new, Riverdale, tonight, ?, u..."
...,...,...,...,...,...,...,...
2740,$ES_F $SPY Bulls are just relentless happy ...,1,"[$, ES_F, $, SPY, Bulls, are, just, relentless...","[$, es_f, $, spi, bull, are, just, relentless,...","[$, ES_F, $, SPY, Bulls, are, just, relentless...","[$, es_f, $, spi, bull, are, just, relentless,...","[$, ES_F, $, SPY, Bulls, are, just, relentless..."
2741,you know that the problem still exist :D,1,"[you, know, that, the, problem, still, exist, ...","[you, know, that, the, problem, still, exist, ...","[you, know, that, the, problem, still, exist, ...","[you, know, that, the, problem, still, exist, ...","[you, know, that, the, problem, still, exist, ..."
2742,- top engaged members this week happy,1,"[-, top, engaged, members, this, week, happy]","[-, top, engag, member, thi, week, happi]","[-, top, engaged, member, this, week, happy]","[-, top, engag, member, thi, week, happi]","[-, top, engaged, member, this, week, happy]"
2743,ngam to weeks left for cadet pilot exam cryin...,1,"[ngam, to, weeks, left, for, cadet, pilot, exa...","[ngam, to, week, left, for, cadet, pilot, exam...","[ngam, to, week, left, for, cadet, pilot, exam...","[ngam, to, week, left, for, cadet, pilot, exam...","[ngam, to, week, left, for, cadet, pilot, exam..."


In [18]:
class TO_VEC:
    def __init__(self, data_frame):
        self.text = data_frame
        self.vectors = pd.DataFrame()

    def to_bin_vec(self):
        coun_vect = CountVectorizer(binary=True)
        for i in range(self.text.shape[1]):
            if i == 1: continue #костыль чтобы игнорить target столбец
            text = pd.DataFrame(map(lambda x: ' '.join(x), self.text.iloc[:,2])).iloc[:, 0].tolist()
            name = self.text.iloc[:,i].name
            count_matrix = coun_vect.fit_transform(text)
            count_array = count_matrix.toarray()
            self.vectors["bin_vec_{}".format(name)] = list(count_array)

    def to_TFIDF(self):
        vectorizer = TfidfVectorizer()
        for i in range(self.text.shape[1]):
            if i == 1: continue #костыль чтобы игнорить target столбец
            text = pd.DataFrame(map(lambda x: ' '.join(x), self.text.iloc[:,2])).iloc[:, 0].tolist()
            name = self.text.iloc[:,i].name
            count_matrix = vectorizer.fit_transform(text)
            count_array = count_matrix.toarray()
            self.vectors["TFIDF_{}".format(name)] = list(count_array)

    def to_word_counts(self):
        coun_vect = CountVectorizer()
        for i in range(self.text.shape[1]):
            if i == 1: continue #костыль чтобы игнорить target столбец
            text = pd.DataFrame(map(lambda x: ' '.join(x), self.text.iloc[:,2])).iloc[:, 0].tolist()
            name = self.text.iloc[:,i].name
            count_matrix = coun_vect.fit_transform(text)
            count_array = count_matrix.toarray()
            self.vectors["word_counts_{}".format(name)] = list(count_array)


    # def to_wordtovec:



In [19]:
vectors = TO_VEC(preproc_data)
vectors.to_bin_vec()
vectors.to_TFIDF()
vectors.to_word_counts()

In [20]:
vectors.vectors

,bin_vec_tweets,bin_vec_just_tokenization,bin_vec_PorterStemmer+misspellings,bin_vec_WordNetLemmatizer+misspellings,bin_vec_PorterStemmer,bin_vec_WordNetLemmatizer,TFIDF_tweets,TFIDF_just_tokenization,TFIDF_PorterStemmer+misspellings,TFIDF_WordNetLemmatizer+misspellings,TFIDF_PorterStemmer,TFIDF_WordNetLemmatizer,word_counts_tweets,word_counts_just_tokenization,word_counts_PorterStemmer+misspellings,word_counts_WordNetLemmatizer+misspellings,word_counts_PorterStemmer,word_counts_WordNetLemmatizer
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0

# TRAIN

In [22]:
X = vectors.vectors.iloc[:,2]
y = preproc_data.category

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    train_size=0.8, 
                                                    random_state=42,
                                                    shuffle = True)

In [72]:
X_train

2226    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1736    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1018    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
891     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2206    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
                              ...                        
1638    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1095    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1130    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1294    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
860     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: bin_vec_PorterStemmer+misspellings, Length: 2196, dtype: object

In [45]:
new_df = pd.DataFrame(['new_values'].str.split('\n',expand=True)
new_df


,bin_vec_PorterStemmer+misspellings
2226,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1736,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1018,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
891,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2206,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...
1638,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1095,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1130,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1294,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [68]:
X_train.array

<PandasArray>
[array([0, 0, 0, ..., 0, 0, 0]), array([0, 0, 0, ..., 0, 0, 0]),
 array([0, 0, 0, ..., 0, 0, 0]), array([0, 0, 0, ..., 0, 0, 0]),
 array([0, 0, 0, ..., 0, 0, 0]), array([0, 0, 0, ..., 0, 0, 0]),
 array([0, 0, 0, ..., 0, 0, 0]), array([0, 0, 0, ..., 0, 0, 0]),
 array([0, 0, 0, ..., 0, 0, 0]), array([0, 0, 0, ..., 0, 0, 0]),
 ...
 array([0, 0, 0, ..., 0, 0, 0]), array([0, 0, 0, ..., 0, 0, 0]),
 array([0, 0, 0, ..., 0, 0, 0]), array([0, 0, 0, ..., 0, 0, 0]),
 array([0, 0, 0, ..., 0, 0, 0]), array([0, 0, 0, ..., 0, 0, 0]),
 array([0, 0, 0, ..., 0, 0, 0]), array([0, 0, 0, ..., 0, 0, 0]),
 array([0, 0, 0, ..., 0, 0, 0]), array([0, 0, 0, ..., 0, 0, 0])]
Length: 2196, dtype: object

In [69]:
pd.DataFrame(list(map(lambda x: [x], X_train)))

,0
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...
2191,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2192,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2193,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2194,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [48]:
from sklearn import svm
clf = svm.SVC()
clf.fit(X_train, y_train)
predict = clf.predict(X_test)
accuracy_score(predict, y_test)

ValueError: setting an array element with a sequence.

In [ ]:
# lr_data = LogisticRegression(multi_class='multinomial', solver='lbfgs')
# lr_data.fit(tfidf_train_data, y_train)
# tfidf_pred_data = lr_data.predict(tfidf_test_data)
# print('test')
# print(classification_report(y_test, tfidf_pred_data))
# print('train')
# print(classification_report(y_train, lr_data.predict(tfidf_train_data)))

# def plot_conf_matrix(true, pred, cat=['-1', '0', '1']):
#     sns.heatmap(data=confusion_matrix(true, pred), 
#             annot=True, fmt="d", cbar=False, xticklabels=cat, yticklabels=cat)


# plot_conf_matrix(y_test, tfidf_pred_data)

# accuracy_score(y_test, tfidf_pred_data)

In [ ]:
#CASHE::



# import nltk
# import ssl

# try:
#     _create_unverified_https_context = ssl._create_unverified_context
# except AttributeError:
#     pass
# else:
#     ssl._create_default_https_context = _create_unverified_https_context

# nltk.download()


# data_neg = list(csv.reader(open("data/processedNegative.csv"), delimiter=','))[0]
# data_pos = list(csv.reader(open("data/processedNeutral.csv"), delimiter=','))[0]
# data_neut = list(csv.reader(open("data/processedPositive.csv"), delimiter=','))[0]

# X = data_neg + data_pos + data_neut
# y = list(np.ones(len(data_neg)) * 0) + list(np.ones(len(data_pos)))  +  list(np.ones(len(data_neut)) * 2)
